In [70]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *

from torch.optim.adam import Adam

In [71]:
criterion1 = nn.MSELoss()

def criterion2(actual, predict):
    
    loss = 0
    div = 0
    
    for i in range(7):
        div += (i+1)
    
    for i in range(32):
        
        for j in range(7):
            loss += (j+1) * (abs(actual[i][j] - predict[i][j]))
              
    loss /= div
    loss /= 32
        
    return loss


def criterion3(actual, predict):
    
    loss = 0
    div = 0
    
    for i in range(7):
        div += (i+1)**2
    
    for i in range(32):
        
        for j in range(7):
            loss += ((j+1)**2) * ((actual[i][j] - predict[i][j])**2)
               
    loss /= div   
    loss = loss**(1/2) 
    loss /= 32
    
    return loss

def mto(data):
    
    df = Prepare_df.processing(data, 'Date', 'alpha')
    x = df.iloc[:, 0:]
    y = df.iloc[:,:1]
    x_ss, y_ms = Prepare_df.scailing(x, y)
    x = x.to_numpy()
    y = y.to_numpy()
    x, y = Prepare_df.window_sliding(df, x, y, 60, 1)
    x_ss, y_ms = Prepare_df.window_sliding(df, x_ss, y_ms, 60, 1)
    x_train = x_ss[:800]
    y_train = y_ms[:800]
    x_test = x_ss[800:]
    y_test = y_ms[800:]

    train = torch.utils.data.TensorDataset(x_train, y_train)
    test = torch.utils.data.TensorDataset(x_test, y_test)

    train_loader = torch.utils.data.DataLoader(dataset = train, batch_size = batch_size, shuffle = False)
    test_loader = torch.utils.data.DataLoader(dataset = test, batch_size = batch_size, shuffle = False)
    
    n = len(train_loader)
    loss_list = []
    
    for epoch in range(num_epochs):
        running_loss = 0.0
    
        for data in train_loader:
            seq, target = data
            out = model(seq)
            loss = criterion(out, target)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        loss_list.append(running_loss/n)
        if (epoch+1) % 100 == 0:
            print('epoch: %d loss: %.4f'%(epoch+1, running_loss/n))
        
        if (epoch % patience == 0) & (epoch != 0):
                    
                if loss_list[epoch-patience] < loss_list[epoch]:
                    print('\n Early Stopping / epoch: %d loss: %.4f'%(epoch+1, running_loss/n))
                
                    break
    
    train_predict = model(x_ss)
    predicted = train_predict.cpu().data.numpy()
    label_y = y_ms.cpu().data.numpy()
    
    predicted = predicted.reshape(1110, 1)
    predicted = ms.inverse_transform(predicted)
    label_y = ms.inverse_transform(label_y)
    
    mape = 100 * np.mean(np.abs((label_y-predicted)/label_y))
                         
    return loss_list, predicted, label_y, mape

def mtm(data):
    
    df = Prepare_df.processing(data, 'Date', 'alpha')
    
    x = df.iloc[:, 0:]
    y = df.iloc[:,:1]
    x_ss, y_ms = Prepare_df.scailing(x, y)
    x = x.to_numpy()
    y = y.to_numpy()
    x, y = Prepare_df.window_sliding(df, x, y, 60, 7)
    x_ss, y_ms = Prepare_df.window_sliding(df, x_ss, y_ms, 60, 7)
    x_train = x_ss[:800]
    y_train = y_ms[:800]
    x_test = x_ss[800:]
    y_test = y_ms[800:]

    train = torch.utils.data.TensorDataset(x_train, y_train)
    test = torch.utils.data.TensorDataset(x_test, y_test)

    train_loader = torch.utils.data.DataLoader(dataset = train, batch_size = batch_size, shuffle = False)
    test_loader = torch.utils.data.DataLoader(dataset = test, batch_size = batch_size, shuffle = False)
    
    n = len(train_loader)
    loss_list = []
    
    for epoch in range(num_epochs):
        running_loss = 0.0
    
        for data in train_loader:
            seq, target = data
            out = model(seq, target, 7, 0.5).to(device)
            loss = criterion(out, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        loss_list.append(running_loss/n)
        if (epoch+1) % 100 == 0:
            print('epoch: %d loss: %.6f'%(epoch+1, running_loss/n))
        
        if (epoch % patience == 0) & (epoch != 0):
            
                if loss_list[epoch-patience] < loss_list[epoch]:
                    print('\n Early Stopping / epoch: %d loss: %.4f'%(epoch+1, running_loss/n))
                
                    break
                    
    train_predict = model(x_ss, y_ms, 7, 0.5)
    predicted = train_predict.cpu().data.numpy()
    label_y = y_ms.cpu().data.numpy()
    

    
    mape = 100 * np.mean(np.abs((true-pred)/true))
                         
    return loss_list, predicted, label_y, mape

In [72]:
path = 'Data/SIR_Origin/*.csv'
dic_files = Load_files.load_files(path, -17, -15)
data = dic_files['서울']
data

,Date,City,Susceptible,Infected,Dead,Recovered,alpha,beta,gamma
0,2020-01-20,서울,9535432,0,0,0,0.000000,0.000000,0.000000
1,2020-01-21,서울,9535432,0,0,0,0.000000,0.000000,0.000000
2,2020-01-22,서울,9535432,0,0,0,0.000000,0.000000,0.000000
3,2020-01-23,서울,9535432,0,0,0,0.000000,0.000000,0.000000
4,2020-01-24,서울,9535431,1,0,0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
1167,2023-04-01,서울,3544282,25006,6383,5959761,0.258321,0.074782,0.000000
1168,2023-04-02,서울,3541881,25537,6383,5961631,0.088766,0.026980,0.000039
1169,2023-04-03,서울,3541039,25689,6384,5962320,0.376110,0.110631,0.000117
1170,2023-04-04,서울,3537451,26432,6387,5965162,0.341943,0.103019,0.000113


In [73]:
data.loc[data['alpha']== 0, 'alpha'] = 1e-6
data

,Date,City,Susceptible,Infected,Dead,Recovered,alpha,beta,gamma
0,2020-01-20,서울,9535432,0,0,0,0.000001,0.000000,0.000000
1,2020-01-21,서울,9535432,0,0,0,0.000001,0.000000,0.000000
2,2020-01-22,서울,9535432,0,0,0,0.000001,0.000000,0.000000
3,2020-01-23,서울,9535432,0,0,0,0.000001,0.000000,0.000000
4,2020-01-24,서울,9535431,1,0,0,0.000001,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
1167,2023-04-01,서울,3544282,25006,6383,5959761,0.258321,0.074782,0.000000
1168,2023-04-02,서울,3541881,25537,6383,5961631,0.088766,0.026980,0.000039
1169,2023-04-03,서울,3541039,25689,6384,5962320,0.376110,0.110631,0.000117
1170,2023-04-04,서울,3537451,26432,6387,5965162,0.341943,0.103019,0.000113


In [74]:
criterions_list = [criterion1, criterion2, criterion3]
learning_rates_list = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
patiences_list = [20, 80, 160, 10000]
hidden_sizes_list = [8, 16, 32, 64]
num_layers_list = [1, 2, 4, 8, 16]
batch_sizes_list = [32, 64, 128, 256]
dropout_list = [0, 0.25, 0.5]
trainers_list = [mto, mtm]



In [75]:
Hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for criterion in criterions_list:
        for patience in patiences_list:
            for num_layers in num_layers_list:
                for batch_sizes in batch_sizes_list:
                    for hidden_size in hidden_sizes_list:    
                        for dropout in dropout_list:
                            Hyperparameter_dict[i] = [lr, criterion, patience, num_layers, batch_sizes, hidden_size, dropout]
                            i += 1
                        
print(i)

14400


In [76]:
k = 0
MAPE = 0

ms = MinMaxScaler()
ss = StandardScaler()

path = 'Data/SIR_Origin/*.csv'
dic_files = Load_files.load_files(path, -17, -15)
data = dic_files['서울']

for i in range(len(Hyperparameter_dict)):
        
    lr = Hyperparameter_dict[i][0]
    criterion = Hyperparameter_dict[i][1]
    patience = Hyperparameter_dict[i][2]
    num_layers = Hyperparameter_dict[i][3]
    batch_size = Hyperparameter_dict[i][4]
    hidden_size = Hyperparameter_dict[i][5]
    dropout = Hyperparameter_dict[i][6]
        
    input_size = 3
    sequence_length = 60
    num_epochs = 10000
    
    model_RNN = RNN(input_size = input_size,
                    hidden_size = hidden_size,
                    sequence_length = sequence_length,
                    num_layers = num_layers, 
                    dropout = dropout, 
                    device = device).to(device)
    
    model = model_RNN
    
    optimizer = Adam(model.parameters(), lr = lr)
        

    print(model)
        
    a, b, c, mape = mto(data)
    print(mape)
    if MAPE > mape:
        MAPE = mape
        models_dict[keys[k]] = [lr, criterion, patience, num_layers, batch_size, hidden_size, dropout, MAPE, loss_list, model]

RNN(
  (rnn): RNN(3, 8, batch_first=True)
  (fc1): Linear(in_features=480, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

 Early Stopping / epoch: 61 loss: 0.0002


NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [53]:
b.shape

(1110, 1)

In [54]:
c.shape

(1110, 1)

In [57]:
c

array([[0.00980425],
       [0.0066669 ],
       [0.02245329],
       ...,
       [0.12537   ],
       [0.11398092],
       [0.        ]], dtype=float32)

In [12]:
input_size = 3
sequence_length = 60
num_epochs = 10000
optimizer = Adam(model.parameters(), lr = lr)

model_RNN = RNN(input_size = input_size,
                hidden_size = hidden_size,
                sequence_length = sequence_length,
                num_layers = num_layers, 
                dropout = dropout, 
                device = device).to(device)

model_LSTM = LSTM(input_size = input_size,
                  hidden_size = hidden_size,
                  sequence_length = sequence_length,
                  num_layers = num_layers, 
                  dropout = dropout, 
                  device = device).to(device)

model_GRU = GRU(input_size = input_size,
                hidden_size = hidden_size,
                sequence_length = sequence_length,
                num_layers = num_layers, 
                dropout = dropout, 
                device = device).to(device)

model_BiRNN = BiRNN(input_size = input_size,
                    hidden_size = hidden_size,
                    sequence_length = sequence_length,
                    num_layers = num_layers, 
                    dropout = dropout, 
                    device = device).to(device)

model_BiLSTM = BiLSTM(input_size = input_size,
                      hidden_size = hidden_size,
                      sequence_length = sequence_length,
                      num_layers = num_layers, 
                      dropout = dropout, 
                      device = device).to(device)

model_BiGRU = BiGRU(input_size = input_size,
                    hidden_size = hidden_size,
                    sequence_length = sequence_length,
                    num_layers = num_layers, 
                    dropout = dropout, 
                    device = device).to(device)

model_seq2seq_RNN = RNN_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)

model_seq2seq_LSTM = LSTM_encoder_decoder(input_size = input_size, 
                                         hidden_size = hidden_size,
                                         num_layers = num_layers, 
                                         dropout = dropout,
                                         device = device).to(device)

model_seq2seq_GRU = GRU_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)

model_seq2seq_BiRNN = BiRNN_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

model_seq2seq_BiLSTM = BiLSTM_encoder_decoder(input_size = input_size, 
                                              hidden_size = hidden_size,
                                              num_layers = num_layers, 
                                              dropout = dropout,
                                              device = device).to(device)

model_seq2seq_BiGRU = BiGRU_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

model_dict = {}

keys = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
        'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

models_list = [model_RNN, model_LSTM, model_GRU, 
              model_BiRNN, model_BiLSTM, model_BiGRU,
              model_seq2seq_RNN, model_seq2seq_LSTM, model_seq2seq_GRU, 
              model_seq2seq_BiRNN, model_seq2seq_BiLSTM, model_seq2seq_BiGRU]

C:\Users\PC\anaconda3\envs\KMU_DA\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
